In [214]:
import numpy as np
import os
import argparse
from GridDataGen.utils.io import *
from GridDataGen.utils.process_network import *
from GridDataGen.utils.config import *
from GridDataGen.utils.stats import *
from GridDataGen.utils.param_handler import *
from GridDataGen.utils.load import *
from pandapower.auxiliary import pandapowerNet
import gc
from datetime import datetime
from tqdm import tqdm
from GridDataGen.utils.topology_perturbation import initialize_generator
import psutil
import shutil
import yaml
from GridDataGen.utils.process_network import process_scenario_contingency



# Load the base config
with open("test_current_computation.yaml", "r") as f:
    base_config = yaml.safe_load(f)

args = NestedNamespace(**base_config)

base_path = os.path.join(args.settings.data_dir, args.network.name, "raw")
if os.path.exists(base_path) and args.settings.overwrite:
    shutil.rmtree(base_path)
os.makedirs(base_path, exist_ok=True)

tqdm_log = open(os.path.join(base_path, "tqdm.log"), "a")
error_log_path = os.path.join(base_path, "error.log")
args_log_path = os.path.join(base_path, "args.log")
node_path = os.path.join(base_path, "pf_node.csv")
edge_path = os.path.join(base_path, "pf_edge.csv")
branch_idx_removed_path = os.path.join(base_path, "branch_idx_removed.csv")
edge_params_path = os.path.join(base_path, "edge_params.csv")
bus_params_path = os.path.join(base_path, "bus_params.csv")
scenarios_csv_path = os.path.join(
    base_path, "scenarios_" + args.load.generator + ".csv"
)
scenarios_plot_path = os.path.join(
    base_path, "scenarios_" + args.load.generator + ".html"
)
scenarios_log = os.path.join(base_path, "scenarios_" + args.load.generator + ".log")

timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
tqdm_log.write(f"\nNew generation started at {timestamp}\n")
with open(error_log_path, "a") as f:
    f.write(f"\nNew generation started at {timestamp}\n")
with open(scenarios_log, "a") as f:
    f.write(f"\nNew generation started at {timestamp}\n")
with open(args_log_path, "a") as f:
    f.write(f"\nNew generation started at {timestamp}\n")
    yaml.dump(base_config, f)

# Load network and scenario data
if args.network.source == "pandapower":
    net = load_net_from_pp(args.network.name)
elif args.network.source == "pglib":
    net = load_net_from_pglib(args.network.name)
elif args.network.source == "file":
    net = load_net_from_file(args.network.name)
else:
    raise ValueError("Invalid grid source!")

network_preprocessing(net)
assert (net.sgen["scaling"] == 1).all(), "Scaling factor >1 not supported yet!"

load_scenario_generator = get_load_scenario_generator(args)
scenarios = load_scenario_generator(net, args.load.scenarios, scenarios_log)
scenarios_df = load_scenarios_to_df(scenarios)
scenarios_df.to_csv(scenarios_csv_path, index=False)
plot_load_scenarios_combined(scenarios_df, scenarios_plot_path)
save_edge_params(net, edge_params_path)
save_bus_params(net, bus_params_path)

# Initialize the topology generator
generator = initialize_generator(
    args.topology_perturbation.type,
    args.topology_perturbation.n_topology_variants,
    args.topology_perturbation.k,
    args.topology_perturbation.elements,
    net,
)

# Initialize data structures
csv_data = []
adjacency_lists = []
branch_idx_removed = []
global_stats = Stats() if not args.settings.no_stats else None

scenario_index = 1

# Process the scenario
net = copy.deepcopy(net)
net.load.p_mw = scenarios[:, scenario_index, 0]
net.load.q_mvar = scenarios[:, scenario_index, 1]
run_opf(net)


net_pf = copy.deepcopy(net)
net_pf = pf_preprocessing(net_pf)

# Generate perturbed topologies
perturbed_topologies = generator.generate(net_pf)

# to simulate contingency, we apply the topology perturbation after OPF
for perturbed_topology in perturbed_topologies:
    try:
        run_pf(perturbed_topology)
    except Exception as e:
        with open(error_log_path, "a") as f:
            f.write(
                f"Caught an exception at scenario {scenario_index} in run_pf function: {e}\n"
            )

            continue

            # TODO 1: What to do when the network does not converge for AC-PF? -> we dont have targets for regression!!

    # Append processed power flow data
    csv_data.extend(pf_post_processing(perturbed_topology))
    adjacency_lists.append(get_adjacency_list(perturbed_topology))
    branch_idx_removed.append(
        get_branch_idx_removed(perturbed_topology._ppc["branch"])
    )
    if not args.settings.no_stats:
        global_stats.update(perturbed_topology)


# Save final data
save_node_edge_data(net, node_path, edge_path, csv_data, adjacency_lists)
# save branch_idx_removed to numpy array
save_branch_idx_removed(branch_idx_removed, branch_idx_removed_path)
if not args.settings.no_stats:
    global_stats.save(base_path)
    plot_stats(base_path)

print("Data generation complete.")






Finding upper limit u .

/Users/albanpuech/grid_data_synthetic/venv/lib/python3.9/site-packages/pandapower/create.py:5049: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



....
OPF did not converge for u=1.200. Using u=1.100 for upper limit
min, max of ref_curve: 0.66, 1.1
l, u: 0.66, 1.1
cutting the load profile (original length: 8760, requested length: 100)


/Users/albanpuech/grid_data_synthetic/venv/lib/python3.9/site-packages/pandas/core/dtypes/astype.py:133: ComplexWarning:

Casting complex values to real discards the imaginary part



Data generation complete.


In [215]:
pf_edge = pd.read_csv("../test_data/case24_ieee_rts/raw/pf_edge.csv")
pf_node = pd.read_csv("../test_data/case24_ieee_rts/raw/pf_node.csv")
branch_idx_removed = pd.read_csv("../test_data/case24_ieee_rts/raw/branch_idx_removed.csv")
edge_params = pd.read_csv("../test_data/case24_ieee_rts/raw/edge_params.csv")
bus_params = pd.read_csv("../test_data/case24_ieee_rts/raw/bus_params.csv")

In [216]:
branch_idx_removed


,scenario,0,1
0,0,2,15


In [217]:
base_kv = bus_params["baseKV"].values
sn_mva = 100


In [218]:
idx_removed =  branch_idx_removed.loc[:, ['0', '1']].values[0]
print("edges removed", idx_removed)

edges removed [ 2 15]


In [219]:
edge_params.drop(idx_removed, inplace=True)

In [220]:
# Extract from-bus and to-bus indices for each branch

f = edge_params["from_bus"].values.astype(np.int32)
t = edge_params["to_bus"].values.astype(np.int32)

# Extract branch admittance coefficients
Yff = edge_params["Yff_r"].values + 1j * edge_params["Yff_i"].values
Yft = edge_params["Yft_r"].values + 1j * edge_params["Yft_i"].values
Ytf = edge_params["Ytf_r"].values + 1j * edge_params["Ytf_i"].values
Ytt = edge_params["Ytt_r"].values + 1j * edge_params["Ytt_i"].values

# Get base voltages for the from and to buses (for kA conversion)
Vf_base_kV = base_kv[f]
Vt_base_kV = base_kv[t]

nl = edge_params.shape[0]
nb = bus_params.shape[0]

# i = [0, 1, ..., nl-1, 0, 1, ..., nl-1], used for constructing Yf and Yt
i = np.hstack([np.arange(nl), np.arange(nl)])

# Construct from-end admittance matrix Yf using the linear combination:
# Yf[b, :] = y_ff_b * e_f + y_ft_b * e_t
Yf = csr_matrix((np.hstack([Yff, Yft]), (i, np.hstack([f, t]))), shape=(nl, nb))
Yt = csr_matrix((np.hstack([Ytf, Ytt]), (i, np.hstack([f, t]))), shape=(nl, nb))

In [221]:
def compute_branch_currents_kA(Yf, Yt, V, Vf_base_kV, Vt_base_kV, sn_mva):

    
    If_pu = Yf @ V  # From-end currents in per-unit (I_f = Y_f V)
    If_kA = np.abs(If_pu) * sn_mva / (np.sqrt(3) * Vf_base_kV)  # Conversion to kA

    # Construct to-end admittance matrix Yt:
    # Yt[b, :] = y_tf_b * e_f + y_tt_b * e_t
    It_pu = Yt @ V  # To-end currents in per-unit (I_t = Y_t V)
    It_kA = np.abs(It_pu) * sn_mva / (np.sqrt(3) * Vt_base_kV)  # Conversion to kA





    return If_kA, It_kA

In [222]:
V = pf_node["Vm"].values * np.exp(1j * pf_node["Va"].values * np.pi / 180)
If_kA, It_kA = compute_branch_currents_kA(Yf, Yt, V, Vf_base_kV, Vt_base_kV, sn_mva)

If_kA, It_kA

(array([0.17546403, 0.25518052, 0.09886943, 0.07348013, 0.23954488,
        0.24327099, 0.23442614, 0.6032624 , 0.22396224, 0.38218623,
        0.28779473, 0.20571786, 0.70133643, 0.54024475, 0.71703685,
        1.04284532, 0.26532923, 0.57841542, 0.57841542, 0.61935974,
        0.88226173, 0.1803906 , 0.54833144, 0.33557769, 0.08544352,
        0.08544352, 0.08675645, 0.08675645, 0.20120267, 0.20120267,
        0.37650682, 0.62511666, 0.375087  , 0.19368163, 0.53213818,
        0.35084715]),
 array([0.17515675, 0.26626711, 0.11247232, 0.07275817, 0.23635455,
        0.24597761, 0.23251819, 0.60400414, 0.23011729, 0.38625903,
        0.28605241, 0.20811571, 0.69733627, 0.54029101, 0.71684839,
        1.04479711, 0.26389178, 0.57942865, 0.57942865, 0.62511662,
        0.88319778, 0.1853832 , 0.54825806, 0.34128208, 0.08725273,
        0.08725273, 0.09041433, 0.09041433, 0.20150013, 0.20150013,
        0.38128567, 1.07311693, 0.64389934, 0.33248679, 0.90463486,
        0.59644006]))

In [223]:
perturbed_topology.res_line['from_bus'] = perturbed_topology.line['from_bus']
perturbed_topology.res_line['to_bus'] = perturbed_topology.line['to_bus']
perturbed_topology.res_line['in_service'] = perturbed_topology.line['in_service']
perturbed_topology.res_trafo['hv_bus'] = perturbed_topology.trafo['hv_bus']
perturbed_topology.res_trafo['lv_bus'] = perturbed_topology.trafo['lv_bus']
perturbed_topology.res_trafo['in_service'] = perturbed_topology.trafo['in_service']


In [224]:
def check_branch_currents(net, f, t,If_kA, It_kA,  idx_removed, decimals=6):
    """
    Compares calculated branch currents and loading with pandapower results.

    Parameters:
    - net: pandapower network
    - f: np.ndarray of shape (n_edges, 2), from and to bus indices
    - t: np.ndarray of shape (n_edges, 2), from and to bus indices
    - If_kA: np.ndarray of calculated from-end branch currents
    - It_kA: np.ndarray of calculated to-end branch currents
    - decimals: int, number of decimal places to round to
    """
    
    # create sets of all (from_bus, to_bus, I_from_kA, I_to_kA) for lines
    set_of_lines = set((line.from_bus, line.to_bus, np.round(line.i_from_ka, decimals), np.round(line.i_to_ka, decimals)) for line in net.res_line.itertuples() if line.in_service)
    # add trafos to the set
    set_of_lines.update((trafo.hv_bus, trafo.lv_bus, np.round(trafo.i_hv_ka, decimals), np.round(trafo.i_lv_ka, decimals)) for trafo in net.res_trafo.itertuples() if trafo.in_service)
    set_of_lines_computed = set((f[line_index], t[line_index], np.round(If_kA[line_index], decimals), np.round(It_kA[line_index], decimals)) for line_index in range(f.shape[0]))
    if not set_of_lines == set_of_lines_computed:
        print(set_of_lines - set_of_lines_computed)
        print(set_of_lines_computed - set_of_lines)
    assert set_of_lines == set_of_lines_computed, "Lines do not match"






In [225]:
check_branch_currents(perturbed_topology, f, t, If_kA, It_kA, idx_removed, decimals=3)

In [226]:
def compute_loading(If_kA, It_kA, Vf_base_kV, Vt_base_kV, rate_a):
    """
    Compute per-branch loading using current magnitudes and branch ratings.

    Parameters:
    - edge_index: np.ndarray of shape (n_edges, 2), each row is [from_bus, to_bus]
    - If_kA: np.ndarray of from-side current magnitudes in kA
    - It_kA: np.ndarray of to-side current magnitudes in kA
    - base_kv: np.ndarray of shape (n_buses,), base voltage in kV per bus
    - edge_attr: np.ndarray of shape (n_edges, >=5), edge features, column 4 = RATE_A

    Returns:
    - loading: np.ndarray of shape (n_edges,), max of from and to side loading
    """

    limitf = rate_a / (Vf_base_kV * np.sqrt(3))
    limitt = rate_a / (Vt_base_kV * np.sqrt(3))

    loadingf = If_kA / limitf
    loadingt = It_kA / limitt

    return np.maximum(loadingf, loadingt)

In [227]:
rate_a = edge_params["rate_a"].values
loading = compute_loading(If_kA, It_kA, Vf_base_kV, Vt_base_kV, rate_a)

In [228]:
def check_loading(net, f,t, loading, decimals=6):
    """
    Compare computed loading against values in net.res_line and net.res_trafo.

    Parameters:
    - net: pandapower network
    - edge_index: np.ndarray of shape (n_edges, 2), mapping lines/transformers to buses
    - loading: np.ndarray of computed loading (shape = number of branches)
    """
    
    # create sets of all (from_bus, to_bus, loading) for lines
    set_of_lines = set((line.from_bus, line.to_bus, np.round(line.loading_percent / 100, decimals)) for line in net.res_line.itertuples() if line.in_service)
    # add trafos to the set
    set_of_lines.update((trafo.hv_bus, trafo.lv_bus, np.round(trafo.loading_percent / 100, decimals)) for trafo in net.res_trafo.itertuples() if trafo.in_service) 
    set_of_lines_computed = set((f[line_index], t[line_index], np.round(loading[line_index], decimals)) for line_index in range(f.shape[0]))
    # assert that the two sets are equal
    assert set_of_lines == set_of_lines_computed, "Lines do not match"
    


In [229]:
check_loading(perturbed_topology, f, t, loading, decimals=3)